# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loan CSV file generated
weather_data_csv = "../output_data/weather_df.csv"
weather_data_df = pd.read_csv(weather_data_csv)
weather_data_df.head(20)

,Unnamed: 0,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,0,katsuura,JP,70.93,93,90,7.47,35.1333,140.3000
1,1,bredasdorp,ZA,69.03,54,97,4.21,-34.5322,20.0403
2,2,arlit,NE,103.91,15,2,6.35,18.7369,7.3853
3,3,shenzhen,CN,87.28,83,93,7.54,22.5455,114.0683
4,4,norilsk,RU,83.43,26,63,14.47,69.3535,88.2027
5,5,santa cruz del sur,CU,74.28,92,39,8.93,20.7147,-77.9956
6,6,mataura,NZ,32.49,93,12,2.77,-46.1927,168.8643
7,7,oranjemund,NaN,57.76,72,4,5.35,-28.5500,16.4333
8,8,albany,US,63.30,91,79,3.00,42.6001,-73.9662
9,9,jamestown,US,78.39,82,0,8.05,42.0970,-79.2353


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data_df[["Latitude", "Longitude"]]
humidity = weather_data_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filter vacation with zero cloudiness
vacation_no_cloud = weather_data_df[weather_data_df["Cloudiness"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Temperature"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Temperature"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Windspeed"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity"] < 60]
# Set Index
hotel_df = perfect_vacation.reset_index()
del hotel_df["index"]
hotel_df

,Unnamed: 0,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,139,carnarvon,AU,71.85,55,0,8.05,-24.8667,113.6333
1,209,lubango,AO,71.67,8,0,9.33,-14.9172,13.4925
2,298,saint george,US,75.79,17,0,4.61,37.1041,-113.5841
3,352,karratha,AU,75.60,55,0,7.72,-20.7377,116.8463
4,357,cabra,ES,72.66,44,0,1.01,37.4725,-4.4421
5,440,yurty,RU,74.86,34,0,2.95,56.0498,97.6348


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
vaca_locations = hotel_df[["Latitude", "Longitude"]]
vaca_humidity = hotel_df["Humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Latitude"]
    lng = hotel_df.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df.to_csv("../output_data/hotel_df.csv")
hotel_df

,Unnamed: 0,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Hotel Name
0,139,carnarvon,AU,71.85,55,0,8.05,-24.8667,113.6333,Hospitality Carnarvon
1,209,lubango,AO,71.67,8,0,9.33,-14.9172,13.4925,Hotel Serra da Chela
2,298,saint george,US,75.79,17,0,4.61,37.1041,-113.5841,Best Western Plus Abbey Inn
3,352,karratha,AU,75.60,55,0,7.72,-20.7377,116.8463,ibis Styles Karratha
4,357,cabra,ES,72.66,44,0,1.01,37.4725,-4.4421,MS Fuente Las Piedras
5,440,yurty,RU,74.86,34,0,2.95,56.0498,97.6348,Gostinitsa Khutorok


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

Figure(layout=FigureLayout(height='420px'))